Я сделал target.csv (Count, IsTrain, IsTest, IsValidation) индекс совпадает с date.csv, пофиксил твою функцию с вычеслением глобальной нумерации недель, два твоих бага пофиксил - простые(видимо ты их случайно заккомитил), со сборкой средних у меня возникли запары, сам делай. Учитывай, что по трейну нужно считать признаки не только для самого трейна, но и для тестовой и валидационной выборки (вот с этим у меня проблемы возникли)

In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import sklearn.cross_validation

In [4]:
train = pd.read_csv('data_original/train.csv')
test = pd.read_csv('data_original/test.csv')
test['Count'] = None
frame = [train, test]
common = pd.concat(frame)
common.Date = common.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
common = common.sort_values(by=['Date'])

In [3]:
common.to_csv('data_original/common.csv', index=None)
train.to_csv('data_original/train.csv', index=None)
test.to_csv('data_original/test.csv', index=None)

# date.csv

In [4]:
date = pd.read_csv('data_original/common.csv')[['Date']]
date.Date = date.Date.apply(lambda x: pd.to_datetime(x))

In [5]:
start_date = date.iloc[0]['Date']
date['Year'] = date.Date.apply(lambda x: x.year)
date['Month'] = date.Date.apply(lambda x: x.month)
date['MonthDay'] = date.Date.apply(lambda x: x.day)
date['WeekDay'] = date.Date.apply(lambda x: x.weekday())

In [6]:
date['YearDay'] = date.Date.apply(lambda x: (x-pd.to_datetime(str(x.year)+'-01-01')).days)

In [7]:
date['ThroughMonth'] = date.Date.apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)

In [8]:
date['ThroughWeek'] = date.Date.apply(lambda x: int( ( (x - start_date).days + start_date.weekday()) / 7))
#теперь нет смещения, это могло вводить неоднозначноть, т.е. одной недели могли противоречить две фичи

In [9]:
date['ThroughDay'] = date.Date.apply(lambda x: (x - start_date).days)

In [10]:
date.to_csv('data_transform/date.csv', index=None)

In [11]:
date.head(25)

,Date,Year,Month,MonthDay,WeekDay,YearDay,ThroughMonth,ThroughWeek,ThroughDay
0,2011-10-06,2011,10,6,3,278,0,0,0
1,2011-10-07,2011,10,7,4,279,0,0,1
2,2011-10-09,2011,10,9,6,281,0,0,3
3,2011-11-15,2011,11,15,1,318,1,6,40
4,2011-12-08,2011,12,8,3,341,2,9,63
5,2011-12-09,2011,12,9,4,342,2,9,64
6,2011-12-10,2011,12,10,5,343,2,9,65
7,2011-12-11,2011,12,11,6,344,2,9,66
8,2011-12-13,2011,12,13,1,346,2,10,68
9,2011-12-14,2011,12,14,2,347,2,10,69


# target.csv

In [12]:
sizeValidationSample = 0.15

In [13]:
target = pd.read_csv('data_original/common.csv')[['Count']]
target['IsTest'] = target.Count
target['IsTrain'] = pd.Series(range(target.shape[0]))
target['IsValidation'] = pd.Series(range(target.shape[0]))

In [14]:
target['IsTest'] = target.IsTest.apply(lambda x: 1 if math.isnan(x) else 0)

In [15]:
withCountValue = target[target.IsTest == 0].index
indexTrain, indexValidation = sklearn.cross_validation.train_test_split(withCountValue, test_size=sizeValidationSample)

In [16]:
target['IsTrain'] = target.IsTrain.apply(lambda x: 1 if ((x in indexTrain) & (target.IsTest[x] == 0)) else 0)
target['IsValidation'] = target.IsValidation.apply(lambda x: 1 if ((x in indexValidation) & (target.IsTest[x] == 0)) else 0)

In [17]:
target.to_csv('data_transform/target.csv', index = None)

In [18]:
target.head(10)

,Count,IsTest,IsTrain,IsValidation
0,8,0,1,0
1,88,0,1,0
2,1,0,1,0
3,1,0,1,0
4,2,0,0,1
5,43,0,1,0
6,10,0,1,0
7,1,0,1,0
8,60,0,1,0
9,NaN,1,0,0


# city.csv


In [19]:
cities = pd.read_csv('data_original/common.csv')[['Point']].Point.unique()
city = pd.DataFrame()
city['Point'] = pd.Series(cities)
city['CityName'] = city.Point.apply(lambda x: '-'.join(x.split('-')[0:-1]))
city_names = city.CityName.unique()
city['CityID'] = city.CityName.apply(lambda x: np.where(city_names == x)[0][0]) 
city['BranchNumber'] = city.Point.apply(lambda x: x.split('-')[-1:][0])
# Москва 1 -> Москва
city['CityName'] = city.CityName.apply(lambda x: ('Москва' if x == 'Москва 1' else x))
city.to_csv('data_transform/city.csv', index = None)

In [20]:
city = pd.read_csv('data_transform/city.csv')
cities = city.CityName.unique()
geo = pd.read_csv('data_transform/geo.csv', names=['CityName', 'lat', 'lng'])
geo.CityName = pd.Series(cities)
city = pd.merge(city, geo, how='left', on='CityName')
city.to_csv('data_transform/city.csv')
city.head(10)

,Point,CityName,CityID,BranchNumber,lat,lng
0,Сыктывкар-1,Сыктывкар,0,1,61.647851,50.833903
1,Сыктывкар-2,Сыктывкар,0,2,61.647851,50.833903
2,Ухта-1,Ухта,1,1,63.567321,53.747159
3,Смоленск-1,Смоленск,2,1,54.790311,32.050366
4,Сарапул-1,Сарапул,3,1,56.453929,53.774218
5,Самара-1,Самара,4,1,53.241504,50.221246
6,Челябинск-1,Челябинск,5,1,55.164442,61.436843
7,Белгород-1,Белгород,6,1,50.599713,36.598262
8,Смоленск-2,Смоленск,2,2,54.790311,32.050366
9,Самара-2,Самара,4,2,53.241504,50.221246


# internal_features.csv

In [6]:
src = pd.read_csv('data_transform/date.csv')
src[['Count', 'Point']] = pd.read_csv('data_original/common.csv')[['Count', 'Point']]
dst = pd.DataFrame() 

In [39]:
#tmp = pd.read_csv('data_transform/target.csv')
#tmp = pd.read_csv('data_transform/target.csv')[(tmp.IsTrain == True) | (tmp.IsValidation == True)]

#trainIndex = tmp.index

In [40]:
#src = pd.read_csv('data_transform/date.csv').iloc[trainIndex]
#src[['Count', 'Point']] = pd.read_csv('data_original/common.csv')[['Count', 'Point']].iloc[trainIndex]
#dst = pd.DataFrame() 

In [10]:
tmp = pd.read_csv('data_transform/target.csv')
validationIndex = tmp[tmp.IsValidation == True].index
src.Count[validationIndex] = None

/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [11]:
src.head(10)

,Date,Year,Month,MonthDay,WeekDay,YearDay,ThroughMonth,ThroughWeek,ThroughDay,Count,Point
0,2011-10-06,2011,10,6,3,278,0,0,0,8.0,Сыктывкар-1
1,2011-10-07,2011,10,7,4,279,0,0,1,88.0,Сыктывкар-1
2,2011-10-09,2011,10,9,6,281,0,0,3,1.0,Сыктывкар-1
3,2011-11-15,2011,11,15,1,318,1,6,40,1.0,Сыктывкар-1
4,2011-12-08,2011,12,8,3,341,2,9,63,NaN,Сыктывкар-1
5,2011-12-09,2011,12,9,4,342,2,9,64,43.0,Сыктывкар-1
6,2011-12-10,2011,12,10,5,343,2,9,65,10.0,Сыктывкар-1
7,2011-12-11,2011,12,11,6,344,2,9,66,1.0,Сыктывкар-1
8,2011-12-13,2011,12,13,1,346,2,10,68,60.0,Сыктывкар-1
9,2011-12-14,2011,12,14,2,347,2,10,69,NaN,Сыктывкар-1


## Mean

### by date independed from points

In [16]:
def calc_mean(src, group_column, target_column):
    mean = src.groupby([group_column])[target_column].mean()
    return src[group_column].apply(lambda x: mean[x])

In [13]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column] = calc_mean(src, column, 'Count')

In [52]:
dst.head(15)

,MeanForMonth,MeanForMonthDay,MeanForThroughDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay
0,111.622891,105.298917,8,32.333333,32.333333,91.608080,83.608696,95.068493
1,111.622891,107.544025,88,32.333333,32.333333,123.291624,83.608696,89.567416
2,111.622891,108.088021,1,32.333333,32.333333,123.320375,83.608696,144.378788
3,103.342567,108.242178,1,1.000000,1.000000,87.412818,83.608696,126.147727
4,106.585433,109.729206,2,96.052632,14.000000,91.608080,83.608696,75.836538
5,106.585433,108.088021,43,96.052632,14.000000,123.291624,83.608696,76.759615
6,106.585433,102.190296,10,96.052632,14.000000,142.669791,83.608696,84.333333
7,106.585433,103.313302,1,96.052632,14.000000,123.320375,83.608696,103.978723
8,106.585433,100.418098,60,96.052632,100.400000,87.412818,83.608696,127.679245
9,106.585433,107.296915,NaN,96.052632,100.400000,87.469894,83.608696,120.674419


### by date depended from points

In [14]:
def map_group_two_key(group, x, y):
    return group[x][y]

def calc_mean_by_columns(src, group_columns, target_column):
    mean = src.groupby(group_columns)[target_column].mean()
    return src[group_columns].apply(lambda x: map_group_two_key(mean, *x), axis=1)

In [15]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column + 'ByPoints'] = calc_mean_by_columns(src, [column, 'Point'], 'Count')

In [17]:
dst.to_csv('data_transform/internal_features.csv')

# All in one

In [18]:
data = pd.read_csv('data_transform/target.csv')
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
internal_features = pd.read_csv('data_transform/internal_features.csv', index_col = 0)

need_columns = date.columns
data[need_columns] = date[need_columns]

need_columns = city.columns
data[need_columns] = city[need_columns]

need_columns = internal_features.columns
data[need_columns] = internal_features[need_columns]


In [19]:
data.to_csv('data_transform/all.csv', index=None)